<h1 style="background: orange; color: #5E7AFF; text-align: center; line-height: 200%; letter-spacing: 1px; font-weight: bold;">Möbius's Companion<br>
<span style="font-size: smaller; font-weight: normal;line-height: 150%;">Peter Luschny, April 2025</span></h1>

In [1]:
# This is a SageMath 10.5 Jupyter notebook (running on Python 3.13).

from typing import Callable
from itertools import accumulate
from sage.all import ZZ, valuation, matrix, srange, cached_function, moebius, simplify, exp, log, divisors, factor, prime_divisors, euler_phi, prime_range, is_prime, sigma, sqrt, prod, is_squarefree, is_prime_power, sign
omega = sloane.A001221
bigomega = sloane.A001222

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Introduction</h1>

 Central to our considerations are four regular integer triangles (lower triangular matrices), T, M, V, H, and two sequences, µ and ν, which are connected by the diagram below.<br> 

The <i>triangles</i> are:
* T(n, k) is the divisibility triangle,
* V(n, k) is the valuation triangle,
* M(n, k) is the Moebius triangle,
* H(n, k) is the Hensel triangle.<br>

The <i>arrows</i> denote<br>
* <i>inv</i> matrix inversion, 
* <i>sign</i> the signum function, and 
* <i>c_1</i> extraction of column 1 from a (0-based) matrix.<br>

The <i>sequences</i> are<br>
* µ the Moebius sequence,
* ν the Hensel sequence.

<img src="DivDiagram.png" width=640 height=540 />

<p>The terms <i>Hensel triangle</i> and <i>Hensel sequence</i> are not standard but are introduced here in honor of Kurt Hensel. Hensel is considered the Father of Valuation Theory.

Divisibility theory starts with the divisibility triangle T = A113704 and its inverse, the Möbius triangle M = A363914, whose first column represents the Möbius function µ = A008683. This theory considers divisibility without taking multiplicity of divisors into account.

The theory that considers the multiplicity of divisors is based on the extended valuation triangle A382944 and its inverse H = A382881, whose first column is the analog of the Möbius function and is given by ν = A382883. 

Just as the values of the Möbius function (0, 1, -1, not 0) lead to the decomposition (A013929, A030229, A030059, and A005117), sequence A382883 leads to the decomposition (A382943, A383016, A383017, and A383106).

The summatory function of ν = A382883 is Σν = A382942, which is the counterpart of Mertens's function Σµ = A002321. The counterpart of A055615 is A382940 = A382883*A000027, and Λν = A382941 is the counterpart of the exponential von Mangoldt function Λµ = A014963.

Similar to the Möbius sequence µ = A008683, sequence ν = A382883 can be used as a transformation. Under this transformation, A000012 is mapped to A383104 and A000027 to A383124. The Dirichlet inverse of A382883 is A383210.

The valuation triangle V = A382944. In addition to divisibility, V(n, k) indicates the <i>order of divisibility</i>. If a term V(n, k) > 1 is replaced by 1 the triangle reduces to the divisibility triangle T = A113704. For n, k >= 2 V(n, k) is defined as the multiplicity of a divisor, i.e., the exponent of the highest order of k that divides n. For a prime number p V(n, p) is called p-adic valuation or p-adic order of n. 

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Notation</h1>

**µ** is the name of the Möbius sequence (A008683). <br>
**ν** is the name of the Hensel sequence (A382883).

**µ** is the Greek small letter mu and ν is the Greek small letter nu.<br>
We also use these Unicode characters as Python names and parts of Python names. 

**ΣT** is the sequence of row sums of the lower triangular matrix T. <br>
**Σa** is the sequence of partial sums of the sequence a. 

The identifiers **T**, **M**, **V**, and **H** are identifiers for the triangles introduced above.

The Iverson bracket **[b]** equals 1 if b is True and 0 if b is False.<br>

<p style="color:brown;font-size:large">Divisibility indicator and valuation:

* For 2 <= k <= n: <br>
**[k | n]**  = 0 if k does not divide n and otherwise is 1.<br>
**[k || n]** = 0 if k does not divide n and otherwise is the largest exponent e such that k^e divides n.<br>
* For k = 1: [k | n] = [k || n] = 1.<br>
* For k = 0: [k | n] = [k || n] = 0^n where 0^0 = 1.<br>

Thus [k | n] = 0 <=> [k || n] = 0 and if [k || n] > 0 then [k | n] = 1 and [k | n] = signum([k || n]).

<p style="color:brown;font-size:large">Concerned with sequences: 

|   | Name                                     |  OEIS                                     | Values   |
|:-: | :-------------------------------         | :-----:                                   | :------  | 
| **T(n, k)<br>[k \| n]** | **The divisibility triangle** | <a href="https://oeis.org/A113704" target="_blank">A113704</a> | 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0 |
| **M(n, k)<br>invT** | **The Moebius triangle**<br> -Sum_{d\|n, d<n} [d \| n]*µ(d, k) | <a href="https://oeis.org/A363914" target="_blank">A363914</a> | 1, 0, 1, 0, -1, 1, 0, -1, 0, 1, 0, 0, -1, 0, 1, 0, -1, 0 |
| **µ(n)** | **The Moebius function**<br> -Sum_{d\|n, d<n} [d \| n]*µ(d)       | <a href="https://oeis.org/A008683" target="_blank">A008683</a> | 1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1 |
| **V(n, k)<br>[k \|\| n]** | **The valuation triangle**  | <a href="https://oeis.org/A382944" target="_blank">A382944</a> | 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 1, 0, 1, 0 |
| **H(n, k)<br>invV** | **The Hensel triangle**<br> -Sum_{d\|n, d<n} [d \|\| n]*ν(d, k) | <a href="https://oeis.org/A382881" target="_blank">A382881</a> | 1, 0, 1, 0, -1, 1, 0, -1, 0, 1, 0, 1, -2, 0, 1, 0, -1 |
| **ν(n)** | **The Hensel function**<br> -Sum_{d\|n, d<n} [d \|\| n]*ν(d)       | <a href="https://oeis.org/A382883" target="_blank">A382883</a> | 1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0 |
| **Σν**  | Sum_{j=1..n} ν(j)                  | <a href="https://oeis.org/A382942" target="_blank">A382942</a> | 1, 0, -1, 0, -1, 0, -1, 0, 1, 2, 1, 1, 0, 1, 2, 2, 1 |
| **Σµ**  | Sum_{j=1..n} µ(j)                  | <a href="https://oeis.org/A002321" target="_blank">A002321</a> | 1, 0, -1, -1, -2, -1, -2, -2, -2, -1, -2, -2, -3, -2 |
|   | Sum_{d\|n} ν(d)                          | <a href="https://oeis.org/A383104" target="_blank">A383104</a> | 1, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 2, 0, 1 |
|   | Sum_{d\|n} ν(n/d)*n                      | <a href="https://oeis.org/A383124" target="_blank">A383124</a> | 1, 1, 2, 3, 4, 2, 6, 7, 7, 4, 10, 7, 12, 6, 8, 14 |
|   | Sum_{d\|n} ν(n/d)*ν(d)                   | <a href="https://oeis.org/A383210" target="_blank">A383210</a> | 1, 1, 1, 0, 1, 1, 1, -2, 0, 1, 1, -1, 1, 1, 1, -3, 1 |
|   | Sum_{d\|n} ν(n/d)*µ(d)                   | <a href="https://oeis.org/A383123" target="_blank">A383123</a> | 1, -2, -2, 2, -2, 4, -2, 0, 2, 4, -2, -3, -2, 4, 4  |
| **Λν** | Exp(Sum_{d\|n} ν(d)*log(n/d))       | <a href="https://oeis.org/A382941" target="_blank">A382941</a> | 1, 2, 3, 2, 5, 1, 7, 4, 3, 1, 11, 3, 13, 1, 1, 16, 17|
| **Λµ** | Exp(Sum_{d\|n} µ(d)*log(n/d))       | <a href="https://oeis.org/A014963" target="_blank">A014963</a> | 1, 2, 3, 2, 5, 1, 7, 2, 3, 1, 11, 1, 13, 1, 1, 2, 17|
|   | {k \| ν(k) = 0}                          | <a href="https://oeis.org/A382943" target="_blank">A382943</a> | 12, 16, 18, 20, 24, 28, 40, 44, 45, 48, 50, 52, 54|
|   | {k \| ν(k) != 0}                         | <a href="https://oeis.org/A383106" target="_blank">A383106</a> | 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 19|
|   | {k \| ν(k) = 1}                          | <a href="https://oeis.org/A383016" target="_blank">A383016</a> | 1, 4, 6, 8, 9, 10, 14, 15, 21, 22, 25, 26, 27, 32|
|   | {k \| ν(k) = -1}                         | <a href="https://oeis.org/A383017" target="_blank">A383017</a> | 2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 30, 31, 36, 37|
|   | {k \| ν(k) != µ(k)}                      | <a href="https://oeis.org/A383105" target="_blank">A383105</a> | 4, 8, 9, 25, 27, 32, 36, 49, 64, 100, 121, 125|
|   | {k \| ν(k) != µ(k) and omega(k) = 1}     | <a href="https://oeis.org/A383211" target="_blank">A383211</a> | 4, 8, 9, 25, 27, 32, 49, 64, 121, 125, 128, 169|
|   | {k \| ν(k) = 1 and µ(k) = 0}             | <a href="https://oeis.org/A383103" target="_blank">A383103</a> | 4, 8, 9, 25, 27, 32, 49, 121, 125, 128, 169, 243|
|   | {k \| ν(k) = -1 and µ(k) = 0}            | <a href="https://oeis.org/A383018" target="_blank">A383018</a> | 36, 64, 100, 196, 216, 225, 441, 484, 676, 729|
|   | ν(n) * n                                 | <a href="https://oeis.org/A382940" target="_blank">A382940</a> | 1, -2, -3, 4, -5, 6, -7, 8, 9, 10, -11, 0, -13, 14, 15|
|   | ν(n) * µ(n)                              | <a href="https://oeis.org/A008966" target="_blank">A008966</a> | 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1 |


This translates the above diagram to relations between OEIS sequences.

<img src="AnumDiagram.png" width=640 height=480 />

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Divisibility</h1>

<p style="color:brown;">Let's start with the divisibility triangle. In Python, we will write T(n, k) instead of [k | n] for the Python function. The number 'n' will always denote a nonnegative integer, n = 0, 1, 2, ... For numbers n,d >= 0, our definition of divisibility is:

$ 
    d \text{ divides } n \Longleftrightarrow n = m \cdot d  \quad \text{for some } m \, \le n.
$

<p style="color:brown;">Our focus is purely on divisibility <i>within</i> the natural numbers where order makes sense, since N_{0} is naturally ordered, and the extra condition is harmless:<br>

• For n > 0, this is exactly the usual definition—and forces 0 out of the list of divisors of any positive n.<br>
• For n = 0, it says "d <= 0", so d = 0, and indeed 0 = 0 · m holds for arbitrary m.<br>

<p style="color:brown;">Thus this both (a) reflects our intuition that "divisors of n" live inside {0, 1, ... , n}, and (b) keeps every divisor-set finite—even for n = 0.
Other possible definitions either leave "divisors of 0" undefined, throw an error, or end up with an infinite set when asked for the divisors of 0. Invoking the order constraint is a mild price to pay for restoring finiteness and intuition.

In [2]:
# The basic query function 'divides' in Sage is part of the
# Integer structure, and we use it unchanged. ZZ(k) simply
# ensures that k is an element of ZZ (the Integers).
# The type conversion int() implements the Iverson bracket.

def T(n: int, k: int) -> int:
    return int(ZZ(k).divides(n))
 
for n in range(13):
    print([n], [T(n, k) for k in range(n + 1)])

[0] [1]
[1] [0, 1]
[2] [0, 1, 1]
[3] [0, 1, 0, 1]
[4] [0, 1, 1, 0, 1]
[5] [0, 1, 0, 0, 0, 1]
[6] [0, 1, 1, 1, 0, 0, 1]
[7] [0, 1, 0, 0, 0, 0, 0, 1]
[8] [0, 1, 1, 0, 1, 0, 0, 0, 1]
[9] [0, 1, 0, 1, 0, 0, 0, 0, 0, 1]
[10] [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1]
[11] [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[12] [0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1]


<p style="color:brown;font-size:large">The Moebius triangle M(n, k) is defined as the inverse of the divisibility triangle. Outside the lower triangular matrix 0 <= k <= n all terms are 0.</p>

In [3]:
m = matrix(ZZ, 10, 10, lambda n, k: k <= n and ZZ(k).divides(ZZ(n)))
m.inverse()

[ 1  0  0  0  0  0  0  0  0  0]
[ 0  1  0  0  0  0  0  0  0  0]
[ 0 -1  1  0  0  0  0  0  0  0]
[ 0 -1  0  1  0  0  0  0  0  0]
[ 0  0 -1  0  1  0  0  0  0  0]
[ 0 -1  0  0  0  1  0  0  0  0]
[ 0  1 -1 -1  0  0  1  0  0  0]
[ 0 -1  0  0  0  0  0  1  0  0]
[ 0  0  0  0 -1  0  0  0  1  0]
[ 0  0  0 -1  0  0  0  0  0  1]

<p style="color:brown;font-size:large">Alternatively we can use the Sage library function 'moebius' to build the triangle.<br>Recall that the operator '//' means <i>integer division</i>, in other languages often called 'div'.</p>

In [4]:
# Alternative:
def M(n: int, k: int) -> int:
    if k == 0: return k^n
    return moebius(n // k) if k.divides(n) else 0

for n in range(10): 
    print([M(n, k) for k in srange(n + 1)])

[1]
[0, 1]
[0, -1, 1]
[0, -1, 0, 1]
[0, 0, -1, 0, 1]
[0, -1, 0, 0, 0, 1]
[0, 1, -1, -1, 0, 0, 1]
[0, -1, 0, 0, 0, 0, 0, 1]
[0, 0, 0, 0, -1, 0, 0, 0, 1]
[0, 0, 0, -1, 0, 0, 0, 0, 0, 1]


<p style="color:brown;font-size:large">Looking at the case k = 1, the defining condition</p>

moebius(n // k) if k.divides(n) else 0

<p style="color:brown;font-size:large">reduces to moebius(n). In other words, column 1 <i>is</i> the Moebius function. Its name in the OEIS is:</p>


In [5]:
µ = moebius
A008683 = µ
print([A008683(n) for n in range(1, 30)])

[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1]


<p style="color:brown;font-size:large">This simple relationship between the Moebius matrix and the Moebius function now serves as a blueprint for our next steps. In this, the valuation triangle will take on the role of the divisibility triangle, and the function <i>v</i>, defined as the first column of the matrix inverse to V, will take on the role of the Moebius function. Our main focus will then be on this function to which we will referre to as the Hensel function (or Hensel sequence).</p>

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Valuation</h1>

<p style="color:brown;font-size:large">We will write V(n, k) instead of [k || n] for the Python function. Outside the lower triangular matrix 0 <= k <= n all terms are 0.</p>

In [6]:
@cached_function
def V(n: int, k: int) -> int:
    if not ZZ(k).divides(n) or k > n: return 0
    if k == n or k == 1: return 1
    return valuation(n, k)

for n in range(13): print([n], [V(n, k) for k in range(n + 1)])

[0] [1]
[1] [0, 1]
[2] [0, 1, 1]
[3] [0, 1, 0, 1]
[4] [0, 1, 2, 0, 1]
[5] [0, 1, 0, 0, 0, 1]
[6] [0, 1, 1, 1, 0, 0, 1]
[7] [0, 1, 0, 0, 0, 0, 0, 1]
[8] [0, 1, 3, 0, 1, 0, 0, 0, 1]
[9] [0, 1, 0, 2, 0, 0, 0, 0, 0, 1]
[10] [0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1]
[11] [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[12] [0, 1, 2, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1]


<p style="color:brown;font-size:large">Note that if a term > 1 in the valuation triangle is replaced by 1 the triangle reduces to the divisibility triangle. T(n, k) = signum(V(n, k)).
In the OEIS you will find the triangle at this A-number:</p>

In [7]:
A382944 = V
print([A382944(n, k) for n in range(8) for k in srange(n+1)])

[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 2, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1]


<p style="color:brown;font-size:large">The row sums are the number of divisors of n, counted with their multiplicity.

In [8]:
def ΣV(n: int) -> int:
    return sum(V(n, k) for k in srange(n + 1))

print([ΣV(n) for n in range(1, 28)])

[1, 2, 2, 4, 2, 4, 2, 6, 4, 4, 2, 7, 2, 4, 4, 9, 2, 7, 2, 7, 4, 4, 2, 10, 4, 4, 6]


<p style="color:brown;font-size:large">In the OEIS this function is registered as:</p>

In [9]:
A169594 = ΣV

<p style="color:brown;font-size:large">The further development follows the same procedure as for the divisibility triangle. There, the inverse triangle is formed, which is the Moebius triangle. The inverse triangle of V, here called the Hensel triangle H, is defined recursively:

In [10]:
@cached_function
def H(n:int, k:int) -> int:
    if n == k: return 1
    if k == 0: return 0^n
    return -sum(V(n, j)*H(j, k) for j in divisors(n)[:-1])

for n in range(13):
    print([n], [H(n, k) for k in srange(n + 1)])

[0] [1]
[1] [0, 1]
[2] [0, -1, 1]
[3] [0, -1, 0, 1]
[4] [0, 1, -2, 0, 1]
[5] [0, -1, 0, 0, 0, 1]
[6] [0, 1, -1, -1, 0, 0, 1]
[7] [0, -1, 0, 0, 0, 0, 0, 1]
[8] [0, 1, -1, 0, -1, 0, 0, 0, 1]
[9] [0, 1, 0, -2, 0, 0, 0, 0, 0, 1]
[10] [0, 1, -1, 0, 0, -1, 0, 0, 0, 0, 1]
[11] [0, -1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
[12] [0, 0, 1, 0, -1, 0, -1, 0, 0, 0, 0, 0, 1]


<p style="color:brown;font-size:large">Let's check that this is indeed the inverse matrix of V: 

In [11]:
M = matrix(ZZ, 13, V).inverse()
print(M)

[ 1  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  1  0  0  0  0  0  0  0  0  0  0  0]
[ 0 -1  1  0  0  0  0  0  0  0  0  0  0]
[ 0 -1  0  1  0  0  0  0  0  0  0  0  0]
[ 0  1 -2  0  1  0  0  0  0  0  0  0  0]
[ 0 -1  0  0  0  1  0  0  0  0  0  0  0]
[ 0  1 -1 -1  0  0  1  0  0  0  0  0  0]
[ 0 -1  0  0  0  0  0  1  0  0  0  0  0]
[ 0  1 -1  0 -1  0  0  0  1  0  0  0  0]
[ 0  1  0 -2  0  0  0  0  0  1  0  0  0]
[ 0  1 -1  0  0 -1  0  0  0  0  1  0  0]
[ 0 -1  0  0  0  0  0  0  0  0  0  1  0]
[ 0  0  1  0 -1  0 -1  0  0  0  0  0  1]


<p style="color:brown;font-size:large">The Hensel triangle and its A-number in the OEIS.

In [12]:
# Slightly optimized by incorporating the valuation function.

@cached_function
def H(n: int, k: int) -> int:
    if n == k: return 1
    if k == 0: return 0^n
    return -sum((1 if j == 1 else valuation(n, j))*H(j, k) for j in divisors(n)[:-1])

A382881 = H
print([A382881(n, k) for n in range(8) for k in srange(n+1)])

[1, 0, 1, 0, -1, 1, 0, -1, 0, 1, 0, 1, -2, 0, 1, 0, -1, 0, 0, 0, 1, 0, 1, -1, -1, 0, 0, 1, 0, -1, 0, 0, 0, 0, 0, 1]


<p style="color:brown;font-size:large">The row sums for n >= 1 give A000007 with offset 1, which is the identity function for Dirichlet multiplication. 

In [13]:
def ΣH(n: int) -> int:
    return sum(H(n, k) for k in srange(n + 1))

print([ΣH(n) for n in range(1, 20)])

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
The Hensel sequence</h1>

<p style="color:brown;font-size:large">The Hensel sequence ν is defined as column 1 of the inverse of the valuation triangle A382944. This corresponds to the definition of the Möbius function as column 1 of the inverse of the divisibility triangle A113704. We have referred to this sequence as Möbius' companion in the title.

In [14]:
# The offical definition:
# v(n) = H(n, 1)
# Implemented by the recursive formula:

@cached_function
def va(n: int) -> int:
    if n < 2: return n
    return -sum(V(n, j)*va(j) for j in divisors(n)[:-1])

# ... or by the alternative definition:
# v(n) = -Sum_{d | n, 1 < d} H(n, d)

@cached_function
def vb(n: int) -> int: 
    if n < 2: return n
    return -sum(H(n, d) for d in divisors(n)[1:])

print([va(n) for n in range(1, 30)])
print([vb(n) for n in range(1, 30)])

[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]
[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]


<p style="color:brown;font-size:large">Optimized by calling the valuation function directly and using the fact that v(p) = -1 if p is prime, this is the final form of v that we will use:

In [15]:
@cached_function
def ν(n: int) -> int: 
    return (n if n < 2 else 
           -1 if is_prime(n) else
           -sum(1 if d == 1 
                else 0 if ν(d) == 0 
                else valuation(n, d)*ν(d) 
            for d in divisors(n)[:-1]))

print([ν(n) for n in range(1, 30)])

[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]


<p style="color:brown;font-size:large">This is how you find the Hensel sequence in the OEIS catalog:

In [16]:
A382883 = ν
print([A382883(n) for n in srange(1, 30)])
print([A382881(n, 1) for n in srange(1, 30)])

[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]
[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]


<p style="color:brown;font-size:large">The three fibers of the Hensel sequence.

In [17]:
def A382943List(upto) -> list[int]: 
    return [n for n in srange(1, upto) if ν(n) == 0]

def A383017List(upto) -> list[int]: 
    return [n for n in srange(1, upto) if ν(n) == -1]

def A383016List(upto) -> list[int]: 
    return [n for n in srange(1, upto) if ν(n) == 1]

print(A382943List(60))
print(A383017List(60))
print(A383016List(60))

[12, 16, 18, 20, 24, 28, 40, 44, 45, 48, 50, 52, 54, 56]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 30, 31, 36, 37, 41, 42, 43, 47, 53, 59]
[1, 4, 6, 8, 9, 10, 14, 15, 21, 22, 25, 26, 27, 32, 33, 34, 35, 38, 39, 46, 49, 51, 55, 57, 58]


<p style="color:brown;font-size:large">... and, for convenience, also the case v(n) != 0:

In [18]:
def A383106List(upto) -> list[int]: 
    return [n for n in srange(1, upto) if ν(n) != 0]

print(A383106List(30))

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 17, 19, 21, 22, 23, 25, 26, 27, 29]


<p style="color:brown;font-size:large">The summatory function of ν is the counterpart of the Mertens function A002321 and has the OEIS-ID A382942.

In [19]:
def Σν(n: int) -> int:
    return sum(ν(k) for k in range(n+1))

A382942 = Σν

print([Σν(n) for n in range(1, 30)])
print([A382942(n) for n in range(1, 30)])

[1, 0, -1, 0, -1, 0, -1, 0, 1, 2, 1, 1, 0, 1, 2, 2, 1, 1, 0, 0, 1, 2, 1, 1, 2, 3, 4, 4, 3]
[1, 0, -1, 0, -1, 0, -1, 0, 1, 2, 1, 1, 0, 1, 2, 2, 1, 1, 0, 0, 1, 2, 1, 1, 2, 3, 4, 4, 3]


<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Classifying the integers with mu and nu</h1>

<p style="color:brown;font-size:large">The pair (µ, ν) can be used to classify the integers, i.e., divide them into mutually exclusive classes. Nine cases are to be considered, and they generate five nonempty classes.</p>

In [20]:
def munu(a: int, b: int, length: int = 60) -> None:
    print([n for n in range(1, length+1) if µ(n) == a and ν(n) == b][:16])

munu( 0,  0, 60)     # A382943 solutions to nu(n) = 0.
munu( 0, -1, 1050)   # A383018
munu( 0,  1, 350)    # A383103
munu( 1,  1, 60)     # A030229 solutions to mu(n) =  1.
munu(-1, -1, 50)     # A030059 solutions to mu(n) = -1.

[12, 16, 18, 20, 24, 28, 40, 44, 45, 48, 50, 52, 54, 56, 60]
[36, 64, 100, 196, 216, 225, 441, 484, 676, 729, 1000, 1024]
[4, 8, 9, 25, 27, 32, 49, 121, 125, 128, 169, 243, 289, 343]
[1, 6, 10, 14, 15, 21, 22, 26, 33, 34, 35, 38, 39, 46, 51, 55]
[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 30, 31, 37, 41, 42, 43]


|           | ν(n) = -1 |  ν(n) = 0 | ν(n) = 1 |
| :-        | :-----:   | :-----:   | :------: |
| µ(n) = -1 | A030059   | [ ]       | [ ]      |
| µ(n) = 0  | A383018   | A382943   | A383103  |
| µ(n) = 1  | [ ]       | [ ]       | A030229  |

<p style="color:brown;font-size:large">This can be represented as a decomposition of the natural numbers.</p>

<img src="Classification.png" width=540 height=240 />

<p style="color:brown;font-size:large">Related sequences:

In [21]:
print([n for n in range(1, 362)  if µ(n) != ν(n)])     # A383105

[4, 8, 9, 25, 27, 32, 36, 49, 64, 100, 121, 125, 128, 169, 196, 216, 225, 243, 289, 343, 361]


<p style="color:brown;font-size:large">Numbers such that µ(n) = ν(n), are similar but not identical to A259444.

In [22]:
print([n for n in range(1, 30) if µ(n) == ν(n)]) 

[1, 2, 3, 5, 6, 7, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 26, 28, 29]


<p style="color:brown;font-size:large">Note that ν(n) = 0 implies µ(n) = 0. In other words, if ν(n) = 0 then n is divisible by the square of a prime.

In [23]:
# This is a test that if v(n) = 0 then µ(n) = 0.

A = [n for n in range(1, 4362) if ν(n) == 0]
set([µ(n) for n in A])

{0}

<p style="color:brown;font-size:large">CONJECTURE: ν(n) = 0  <=> n is A059404 or n is A217261.<br>
A059404 are the numbers with different exponents in their prime factorization.

In [24]:
# Test that if n is in A059404 then v(n) = 0. 

def isA059404(n) -> bool: 
    return 1 < len(set(valuation(n, p) for p in prime_divisors(n)))

A = [n for n in range(1, 190) if isA059404(n)]
set([ν(n) for n in A])

{0}

<p style="color:brown;font-size:large">A217261 are the numbers of the form i^j^k, for i, j, k > 1.

In [25]:
# Test that if n is in A217261 then v(n) = 0. 

B = [16, 81, 256, 512, 625, 1296, 2401, 4096, 6561, 10000, 14641, 19683, 20736, 
     28561, 38416, 50625, 65536, 83521, 104976, 130321, 160000, 194481, 234256, 
     262144, 279841, 331776, 390625, 456976, 531441, 614656, 707281, 810000, 
     923521, 1048576, 1185921]

print([sqrt(n) for n in B[:20]])  # See A188915 Union of squares and powers of 2.
set([ν(n) for n in B])

[4, 9, 16, 16*sqrt(2), 25, 36, 49, 64, 81, 100, 121, 81*sqrt(3), 144, 169, 196, 225, 256, 289, 324, 361]


{0}

<p style="color:brown;font-size:large">A053810 are the numbers of the form p^e where both p and e are prime numbers.<br>CONJECTURE: If n is in A053810 then ν(n) = 1.

In [26]:
# Test that if n is in A053810 then v(n) = 1. 

def isA053810(n: int) -> bool:
    p = prime_divisors(n)
    return len(p) == 1 and is_prime(valuation(n, p[0]))

S = set([ν(n) for n in range(1, 20000) if isA053810(n)])
print(S)

{1}


<p style="color:brown;font-size:large">Illustrating the different indicator functions:

In [27]:
def indicators(n: int) -> str:
    f = factor(n)
    return f"{n:>4} | {ν(n):>2} | {moebius(n):>2} | {int(isA053810(n)):>2} | {int(isA059404(n)):>2} | {str(f):<12}"

for n in range(1, 20):
    print(indicators(n))
    
print(indicators(99))
print(indicators(127))
print(indicators(2197))

   1 |  1 |  1 |  0 |  0 | 1           
   2 | -1 | -1 |  0 |  0 | 2           
   3 | -1 | -1 |  0 |  0 | 3           
   4 |  1 |  0 |  1 |  0 | 2^2         
   5 | -1 | -1 |  0 |  0 | 5           
   6 |  1 |  1 |  0 |  0 | 2 * 3       
   7 | -1 | -1 |  0 |  0 | 7           
   8 |  1 |  0 |  1 |  0 | 2^3         
   9 |  1 |  0 |  1 |  0 | 3^2         
  10 |  1 |  1 |  0 |  0 | 2 * 5       
  11 | -1 | -1 |  0 |  0 | 11          
  12 |  0 |  0 |  0 |  1 | 2^2 * 3     
  13 | -1 | -1 |  0 |  0 | 13          
  14 |  1 |  1 |  0 |  0 | 2 * 7       
  15 |  1 |  1 |  0 |  0 | 3 * 5       
  16 |  0 |  0 |  0 |  0 | 2^4         
  17 | -1 | -1 |  0 |  0 | 17          
  18 |  0 |  0 |  0 |  1 | 2 * 3^2     
  19 | -1 | -1 |  0 |  0 | 19          
  99 |  0 |  0 |  0 |  1 | 3^2 * 11    
 127 | -1 | -1 |  0 |  0 | 127         
2197 |  1 |  0 |  1 |  0 | 13^3        


<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Von Mangoldt joins the party</h1>

In [28]:
def νMangoldt(n: int) -> int:
    return simplify(exp(sum(ν(d)*log(n//d) for d in divisors(n))))

def µMangoldt(n: int) -> int:
    return simplify(exp(sum(µ(d)*log(n//d) for d in divisors(n))))

In [29]:
print([νMangoldt(n) for n in srange(1, 30)])    # A014963
print([µMangoldt(n) for n in srange(1, 30)])    # A382941

[1, 2, 3, 2, 5, 1, 7, 4, 3, 1, 11, 3, 13, 1, 1, 16, 17, 2, 19, 5, 1, 1, 23, 18, 5, 1, 9, 7, 29]
[1, 2, 3, 2, 5, 1, 7, 2, 3, 1, 11, 1, 13, 1, 1, 2, 17, 1, 19, 1, 1, 1, 23, 1, 5, 1, 3, 1, 29]


<p style="color:brown;font-size:large">Using the product rule for the logarithm the function µMangoldt could also be defined as:

In [30]:
def µM(n: int) -> int:
    return simplify(exp(-sum(µ(d)*log(d) for d in divisors(n))))

print([µM(n) for n in srange(1, 30)])

[1, 2, 3, 2, 5, 1, 7, 2, 3, 1, 11, 1, 13, 1, 1, 2, 17, 1, 19, 1, 1, 1, 23, 1, 5, 1, 3, 1, 29]


<p style="color:brown;font-size:large">But when you try the same trick with the ν function this will not work:

In [31]:
def νM(n: int) -> int:
    return simplify(exp(-sum(ν(d)*log(d) for d in divisors(n))))

print([νM(n) for n in srange(1, 25)])

[1, 2, 3, 1/2, 5, 1, 7, 1/16, 1/3, 1, 11, 1/4, 13, 1, 1, 1/16, 17, 1/9, 19, 1/4, 1, 1, 23, 1/32]


In [32]:
print([n for n in srange(1, 60) if νM(n) >  1])  # 
print([n for n in srange(1, 30) if νM(n) >= 1])  # NOT A005117!
print([n for n in srange(1, 50) if νM(n) <  1])  # A013929
print([n for n in srange(1, 50) if νM(n) == 1])  # 
print("---")
# Compare with other sequences at 432!
print(νMangoldt(432), νM(432), [factor(432)], [factor(186624)], 432^2)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59]
[1, 2, 3, 5, 6, 7, 10, 11, 13, 14, 15, 17, 19, 21, 22, 23, 26, 29]
[4, 8, 9, 12, 16, 18, 20, 24, 25, 27, 28, 32, 40, 44, 45, 48, 49]
[1, 6, 10, 14, 15, 21, 22, 26, 30, 33, 34, 35, 36, 38, 39, 42, 46]
---
186624 1 [2^4 * 3^3] [2^8 * 3^6] 186624


<p style="color:brown;font-size:large">On OEIS use the names:

In [33]:
A382941 = νMangoldt
A014963 = µMangoldt

In [34]:
# Complement of the prime powers A246655 in the positive integers:
print([n for n in srange(1, 40) if µMangoldt(n) == 1])  # A361102 = {1} + A024619 

# Nonprime squarefree numbers:
print([n for n in srange(1, 60) if νMangoldt(n) == 1])  # A000469 = {1} + A120944 

[1, 6, 10, 12, 14, 15, 18, 20, 21, 22, 24, 26, 28, 30, 33, 34, 35, 36, 38, 39]
[1, 6, 10, 14, 15, 21, 22, 26, 30, 33, 34, 35, 38, 39, 42, 46, 51, 55, 57, 58]


<p style="color:brown;font-size:large">One of the (many) reasons why A246655 are THE PRIME POWERS 💪:

In [35]:
print([n for n in srange(1, 40) if µMangoldt(n) != 1])  # A246655
print([n for n in srange(1, 30) if νMangoldt(n) != 1])  # A383263

[2, 3, 4, 5, 7, 8, 9, 11, 13, 16, 17, 19, 23, 25, 27, 29, 31, 32, 37]
[2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 27, 28, 29]


<p style="color:brown;font-size:large">isA383263(n) <=> νMangoldt(n) != 1 <=> is_prime_power(n) or not is_squarefree(n) <=> is_prime(n) or not is_squarefree(n).

In [36]:
def isA383263(n: int) -> bool:
    return is_prime(n) or not is_squarefree(n)

print([n for n in range(1, 30) if isA383263(n)])
print([n for n in range(1, 30) if is_prime_power(n) or not is_squarefree(n)])

[2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 27, 28, 29]
[2, 3, 4, 5, 7, 8, 9, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 27, 28, 29]


<p style="color:brown;font-size:large">Now let's restrict the summation to prime divisors. 

In [37]:
def νpMangoldt(n: int) -> int:
    return simplify(exp(-sum(ν(d)*log(n//d) for d in prime_divisors(n))))

def µpMangoldt(n: int) -> int:
    return simplify(exp(-sum(moebius(d)*log(n//d) for d in prime_divisors(n))))

In [38]:
print([νpMangoldt(n) for n in srange(1, 25)])
print([µpMangoldt(n) for n in srange(1, 25)])

[1, 1, 1, 2, 1, 6, 1, 4, 3, 10, 1, 24, 1, 14, 15, 8, 1, 54, 1, 40, 21, 22, 1, 96]
[1, 1, 1, 2, 1, 6, 1, 4, 3, 10, 1, 24, 1, 14, 15, 8, 1, 54, 1, 40, 21, 22, 1, 96]


<p style="color:brown">We see νpMangoldt = µpMangoldt. Moreover, this sequence was introduced by yours truly in Feb 2012 to the OEIS. In exactly this meaning, as you can see from the first sentence of the commentary, which was the original name. According to a formula from Charles R Greathouse IV we also have:</p>
<p style="color:brown;font-size:large">νpMangoldt(n) = µpMangoldt(n) = n^omega(n)/rad(n) = A205959(n).</p>

In [39]:
def A205959(n: int) -> int:
    return n^omega(n)/ZZ(n).radical()

print([A205959(n) for n in srange(1, 25)])

# def A205959(n): return prod(n//p for p in prime_divisors(n)) 

def A383438(n: int) -> int:
    return sum([A205959(k) for k in srange(1, n+1)])
print([A383438(n) for n in range(1, 22)])

[1, 1, 1, 2, 1, 6, 1, 4, 3, 10, 1, 24, 1, 14, 15, 8, 1, 54, 1, 40, 21, 22, 1, 96]
[1, 2, 3, 5, 6, 12, 13, 17, 20, 30, 31, 55, 56, 70, 85, 93, 94, 148, 149, 189, 210]


<p style="color:brown;font-size:large">Look at the logarithmic scatterplot of A205959 that reveals an interesting shell model.

In [40]:
def A383438List(len: int) -> list[int]:
    return list(accumulate([prod(n/p for p in prime_divisors(n)) for n in range(1, len+1)]))

print(A383438List(23))

[1, 2, 3, 5, 6, 12, 13, 17, 20, 30, 31, 55, 56, 70, 85, 93, 94, 148, 149, 189, 210, 232, 233]


<p style="color:brown;font-size:large">Summatory functions:

In [41]:
def ΣνMangoldt(len: int) -> list[int]:
    return list(accumulate([νMangoldt(n) for n in srange(1, len)]))

def ΣµMangoldt(len: int) -> list[int]:
    return list(accumulate([µMangoldt(n) for n in srange(1, len)]))

In [42]:
print(ΣνMangoldt(22))
print(ΣµMangoldt(22))

[1, 3, 6, 8, 13, 14, 21, 25, 28, 29, 40, 43, 56, 57, 58, 74, 91, 93, 112, 117, 118]
[1, 3, 6, 8, 13, 14, 21, 23, 26, 27, 38, 39, 52, 53, 54, 56, 73, 74, 93, 94, 95]


<p style="color:brown;font-size:large">On OEIS you see this:

In [43]:
missing = ΣνMangoldt
A072107 = ΣµMangoldt

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Transforms à la Dirichlet</h1>

<p style="color:brown;font-size:large">Notation for tranforms:

Given the arithmetic function $ \alpha $, we call the Dirichlet convolution $ \alpha * g $ the $ \alpha $-transform and write

$
   \quad \quad \alpha T : (g, n) \mapsto \sum_{d \mid n} \alpha\!\left(\frac{n}{d}\right) \, g(d).
$

Thus in particular **µT** is the Möbius transform and **νT** its companion transform.

In [44]:
def vT(b, n) -> int:
    return sum(ν(n//d)*b(d) for d in divisors(n))

def µT(b, n) -> int:
    return sum(µ(n//d)*b(d) for d in divisors(n))

<p style="color:brown;font-size:large">Three classical examples for the transforms:

* U: n -> 1 <br>
* N: n -> n <br>
* I: n -> 1 if n = 1 else 0

In [45]:
U = lambda n: 1

print([vT(U, n) for n in srange(1, 30)])  # A383104
print([µT(U, n) for n in srange(1, 30)])  # A000007

[1, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 2, 1, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


<p style="color:brown;font-size:large">On the OEIS you see this: A383104, similar to A366988; apparently with the same indicator function for n > 1.

In [46]:
N = lambda n: n

print([vT(N, n) for n in srange(1, 30)])  # A383124
print([µT(N, n) for n in srange(1, 30)])  # A000010

[1, 1, 2, 3, 4, 2, 6, 7, 7, 4, 10, 7, 12, 6, 8, 14, 16, 8, 18, 13, 12, 10, 22, 17, 21, 12, 22, 19, 28]
[1, 1, 2, 2, 4, 2, 6, 4, 6, 4, 10, 4, 12, 6, 8, 8, 16, 6, 18, 8, 12, 10, 22, 8, 20, 12, 18, 12, 28]


<p style="color:brown;font-size:large">This is A383124 in the case ν and Euler's totient function A000010 in the case µ.<br>
The first sequence is missing in OEIS and the second is µ A008683.

In [47]:
I = lambda n: 1 if n == 1 else 0

print([vT(I, n) for n in srange(1, 30)])  # missing
print([µT(I, n) for n in srange(1, 30)])  # A008683

[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]
[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1]


<p style="color:brown;font-size:large">We now implement the transformations in a more elegant way as sequence-to-sequence transformations:

In [48]:
def νT(b: Callable[[int], int]) -> Callable[[int], int]:
    def νb(n: int) -> int:
        return sum(ν(n//d)*b(d) for d in divisors(n))
    return νb

In [49]:
νU = νT(U) 
νN = νT(N) 
νI = νT(I)

print([νU(n) for n in range(1, 30)])  # A383104
print([νN(n) for n in range(1, 30)])  # A383124
print([νI(n) for n in range(1, 30)])  # A382883

[1, 0, 0, 1, 0, 0, 0, 2, 1, 0, 0, 1, 0, 0, 0, 2, 0, 1, 0, 1, 0, 0, 0, 2, 1, 0, 2, 1, 0]
[1, 1, 2, 3, 4, 2, 6, 7, 7, 4, 10, 7, 12, 6, 8, 14, 16, 8, 18, 13, 12, 10, 22, 17, 21, 12, 22, 19, 28]
[1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 1, 1, 1, 0, -1]


In [50]:
def µT(b: Callable[[int], int]) -> Callable[[int], int]:
    def µb(n: int) -> int:
        return sum(µ(n//d)*b(d) for d in divisors(n))
    return µb

In [51]:
µU = µT(U) 
µN = µT(N) 
µI = µT(I)

print([µU(n) for n in range(1, 30)])  # A000007
print([µN(n) for n in range(1, 30)])  # A000010
print([µI(n) for n in range(1, 30)])  # A008683

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 2, 2, 4, 2, 6, 4, 6, 4, 10, 4, 12, 6, 8, 8, 16, 6, 18, 8, 12, 10, 22, 8, 20, 12, 18, 12, 28]
[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1]


<p style="color:brown;font-size:large;">Let's check some other special cases:</p>

In [52]:
νµ = νT(µ)
µν = µT(ν)

print([νµ(n) for n in range(1, 30)]) 
print([µν(n) for n in range(1, 30)]) 

[1, -2, -2, 2, -2, 4, -2, 0, 2, 4, -2, -3, -2, 4, 4, -1, -2, -3, -2, -3, 4, 4, -2, 0, 2, 4, 0, -3, -2]
[1, -2, -2, 2, -2, 4, -2, 0, 2, 4, -2, -3, -2, 4, 4, -1, -2, -3, -2, -3, 4, 4, -2, 0, 2, 4, 0, -3, -2]


<p style="color:brown;font-size:large;">This is not surprising as the Dirichlet convolution is commutative but we are pleased that it also applies in our case (since we computed them on two different paths).

$$
     \nu * \mu = \mu * \nu  
$$

$$
νT(µ) = µT(ν)
$$

$$
    \sum_{d \mid n} \nu(d) \, \mu\!\left(\frac{n}{d}\right) =
    \sum_{d \mid n} \mu(d) \, \nu\!\left(\frac{n}{d}\right)
$$

<p style="color:brown;font-size:large">In the OEIS it has the form A383123:

In [53]:
@cached_function
def A382883(n: int) -> int: 
    return n if n < 2 else -sum((1 if j == 1 
                                 else valuation(n, j))*A382883(j) 
                            for j in divisors(n)[:-1])

def vT(b: Callable[[int], int]) -> Callable[[int], int]:
    @cached_function
    def vb(n: int) -> int:
        return sum(A382883(n//d)*b(d) for d in divisors(n))
    return vb

A383123 = vT(moebius)
print([A383123(n) for n in range(1, 30)])

[1, -2, -2, 2, -2, 4, -2, 0, 2, 4, -2, -3, -2, 4, 4, -1, -2, -3, -2, -3, 4, 4, -2, 0, 2, 4, 0, -3, -2]


<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
The Dirichlet Inverse</h1>

In [54]:
def iT(b: Callable[[int], int]) -> Callable[[int], int]:
    """We assume b(1) = 1!"""
    @cached_function
    def g(n: int) -> int:
        if n == 1:
            return 1
        s = sum(g(n//d)*b(d) for d in divisors(n)[1:])
        return -s
    return g

<p style="color:brown;font-size:large">Let's make sure that the function does what it should.

In [55]:
print([iT(euler_phi)(n) for n in range(1, 30)])  # A023900
print([iT(sigma)(n) for n in range(1, 30)])      # A046692

[1, -1, -2, -1, -4, 2, -6, -1, -2, 4, -10, 2, -12, 6, 8, -1, -16, 2, -18, 4, 12, 10, -22, 2, -4, 12, -2, 6, -28]
[1, -3, -4, 2, -6, 12, -8, 0, 3, 18, -12, -8, -14, 24, 24, 0, -18, -9, -20, -12, 32, 36, -24, 0, 5, 42, 0, -16, -30]


<p style="color:brown;font-size:large">Dirichlet inverse of positive integers:

In [56]:
def A382940(n:int) -> int: 
    return ν(n) * n

def A055615(n:int) -> int: 
    return µ(n) * n

print([A382940(n) for n in srange(1, 30)])
print([A055615(n) for n in srange(1, 30)])

[1, -2, -3, 4, -5, 6, -7, 8, 9, 10, -11, 0, -13, 14, 15, 0, -17, 0, -19, 0, 21, 22, -23, 0, 25, 26, 27, 0, -29]
[1, -2, -3, 0, -5, 6, -7, 0, 0, 10, -11, 0, -13, 14, 15, 0, -17, 0, -19, 0, 21, 22, -23, 0, 0, 26, 0, 0, -29]


<p style="color:brown;font-size:large">The Dirichlet inverse of ν and the Dirichlet inverse of µ.

In [57]:
print([iT(ν)(n) for n in range(1, 30)])  # A383210
print([iT(µ)(n) for n in range(1, 30)])  # A000012

[1, 1, 1, 0, 1, 1, 1, -2, 0, 1, 1, -1, 1, 1, 1, -3, 1, -1, 1, -1, 1, 1, 1, -5, 0, 1, -2, -1, 1]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


<p style="color:brown;font-size:large"> A383210 / A000012 / A382883 /  A008683

In [58]:
print([iT(U)(n) for n in range(1, 30)])  # A008683
print([iT(N)(n) for n in range(1, 30)])  # A055615
print([iT(I)(n) for n in range(1, 30)])  # A000007

[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0, 1, 1, -1, 0, 0, 1, 0, 0, -1]
[1, -2, -3, 0, -5, 6, -7, 0, 0, 10, -11, 0, -13, 14, 15, 0, -17, 0, -19, 0, 21, 22, -23, 0, 0, 26, 0, 0, -29]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


<p style="color:brown;font-size:large"> A008683 / A055615 / A000007</p>

<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
... and what about primes?</h1>

<p style="color:brown;font-size:large">A000040 The prime numbers.

In [59]:
F = [p for p in prime_range(50)]

print([µ(p) for p in F])
print([ν(p) for p in F])
print([H(p, 1) for p in F])

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


<p style="color:brown;font-size:large">Consider ν(n) = -1 and not is_prime(n) and compare with A325264.

In [60]:
print([n for n in range(100, 223) if ν(n) == -1 and not is_prime(n)])

# 216 is not in A325264
print([100, 102, 105, 110, 114, 130, 138, 154, 165, 170, 174, 182, 
       186, 190, 195, 196, 222])

[100, 102, 105, 110, 114, 130, 138, 154, 165, 170, 174, 182, 186, 190, 195, 196, 216, 222]
[100, 102, 105, 110, 114, 130, 138, 154, 165, 170, 174, 182, 186, 190, 195, 196, 222]


<p style="color:brown;font-size:large">A001248 Squares of primes.

In [61]:
F = [p^2 for p in prime_range(50)]

print([µ(p) for p in F])
print([ν(p) for p in F])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


<p style="color:brown;font-size:large">A246655 Prime powers: numbers of the form p^e where p is a prime and e >= 1.

In [62]:
def A246655List(u: int) -> list[int]: 
    return [n for n in range(1, u) if omega(n) == 1]

LA246655 = A246655List(55)
µA246655 = [µ(p) for p in LA246655] 
νA246655 = [ν(n) for n in LA246655] 

print(LA246655)
print(µA246655)  # This sequence should be in the OEIS!
print(νA246655)  # This sequence should be in the OEIS!

[2, 3, 4, 5, 7, 8, 9, 11, 13, 16, 17, 19, 23, 25, 27, 29, 31, 32, 37, 41, 43, 47, 49, 53]
[-1, -1, 0, -1, -1, 0, 0, -1, -1, 0, -1, -1, -1, 0, 0, -1, -1, 0, -1, -1, -1, -1, 0, -1]
[-1, -1, 1, -1, -1, 1, 1, -1, -1, 0, -1, -1, -1, 1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1]


In [63]:
LA246655 = A246655List(1222) 
print([p for p in LA246655 if ν(p) == 1])  # A053810
print([p for p in LA246655 if ν(p) == 0])  # missing

[4, 8, 9, 25, 27, 32, 49, 121, 125, 128, 169, 243, 289, 343, 361, 529, 841, 961]
[16, 81, 256, 512, 625]


<p style="color:brown;font-size:large">The valuation of the smallest prime divisor of n, A067029. 

In [64]:
# vSPD: the valuation of the smallest prime divisor of n.
# We do not need the full factorization of n, trial division is faster.

def vSPD(n: int) -> int:
    if n < 2: return 1
    if is_prime(n): return 1
    for p in prime_range(2, n):
        if n % p == 0:
            return valuation(n, p)
    return -1 # Will not happen by PNT.
print([vSPD(n) for n in range(1, 30)])

# Alternative:
def vspd(n: int) -> int: return factor(n)[0][1] if n > 1 else 1
print([vspd(n) for n in range(1, 30)])

[1, 1, 1, 2, 1, 1, 1, 3, 2, 1, 1, 2, 1, 1, 1, 4, 1, 1, 1, 2, 1, 1, 1, 3, 2, 1, 3, 2, 1]
[1, 1, 1, 2, 1, 1, 1, 3, 2, 1, 1, 2, 1, 1, 1, 4, 1, 1, 1, 2, 1, 1, 1, 3, 2, 1, 3, 2, 1]


<p style="color:brown;font-size:large">A383264 Numbers whose vSPD is not squarefree, where vSPD(n) is the valuation of the smallest prime divisor of n.

In [65]:
def isA383264(n: int) -> bool: return not is_squarefree(vSPD(n))
print([n for n in range(1, 500) if isA383264(n)])

[16, 48, 80, 81, 112, 144, 176, 208, 240, 256, 272, 304, 336, 368, 400, 405, 432, 464, 496]


<p style="color:brown;font-size:large">A383211 Numbers of the form p^e where p is prime and e > 1 is squarefree.

In [66]:
def A383266(n: int, k: int) -> int:
    if k == 0: return n^2
    if k == 1: return n
    e = 1
    while k^e <= n:
        e += 1
    return e - 1

def A383211List(upto: int) -> list[int]:
    L = []
    for p in prime_range(2, upto + 1):
        E = A383266(upto, p)
        for e in range(2, E+1):
            if is_squarefree(e):
                n = p^e
                if n <= upto:
                    L.append(n)
    return sorted(L)

B = A383211List(1100)
print(B)
print(set([vSPD(n) for n in B]))

[4, 8, 9, 25, 27, 32, 49, 64, 121, 125, 128, 169, 243, 289, 343, 361, 529, 729, 841, 961, 1024]
{2, 3, 5, 6, 7, 10}


<p style="color:brown;font-size:large">Characterisation: isA383211(n) <=> omega(n) = 1 and µ(n) != ν(n).

In [67]:
print([n for n in range(1, 1100) if omega(n) == 1 and µ(n) != ν(n)]) 
A = A246655List(1100)
print([n for n in A if µ(n) != ν(n)])

[4, 8, 9, 25, 27, 32, 49, 64, 121, 125, 128, 169, 243, 289, 343, 361, 529, 729, 841, 961, 1024]
[4, 8, 9, 25, 27, 32, 49, 64, 121, 125, 128, 169, 243, 289, 343, 361, 529, 729, 841, 961, 1024]


<p style="color:brown;font-size:large">The relation to the squarefree numbers A005117.

In [68]:
# 900 is the first number that does not have the form p^e where both p and e are prime numbers.
print([n for n in range(1, 1400) if ν(n) == 1 and not is_squarefree(n)])

[4, 8, 9, 25, 27, 32, 49, 121, 125, 128, 169, 243, 289, 343, 361, 529, 841, 900, 961, 1331, 1369]


<p style="color:brown;font-size:large">The next sequence is similar but different from A259183.


In [69]:
F = [n for n in srange(1, 300) if µ(n) != ν(n)]
print(F)  # This sequence should be in the OEIS!

[4, 8, 9, 25, 27, 32, 36, 49, 64, 100, 121, 125, 128, 169, 196, 216, 225, 243, 289]


In [70]:
F = [bigomega(n) for n in srange(1, 28)]

print([µ(n) for n in F])
print([ν(n) for n in F])

[0, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1, 0, 1, -1, 1, -1, -1, -1, 1, 0, -1, -1, -1]
[0, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, 1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1]


<p style="color:brown;font-size:large">Similar but different from A323350 are the numbers n such that µ(Ω(n)) != ν(Ω(n)).

In [71]:
print([n for n in srange(1, 150) if µ(bigomega(n)) != ν(bigomega(n))])

[16, 24, 36, 40, 54, 56, 60, 81, 84, 88, 90, 100, 104, 126, 132, 135, 136, 140]


<p style="color:brown;font-size:large">µ(τ(n)) != ν(τ(n)). Positive integers whose number of divisors is a perfect power A120497 ?


In [72]:
def tau(n: int) -> int: 
    return sigma(n, 0)

print([n for n in srange(1, 50) if µ(tau(n)) != ν(tau(n))])

[6, 8, 10, 14, 15, 21, 22, 24, 26, 27, 30, 33, 34, 35, 36, 38, 39, 40, 42, 46]


<p style="color:brown;font-size:large">Subsequence of A065496, the numbers n such that sigma(n) is a nontrivial power? 

In [73]:
def s1(n: int) -> int: 
    return sigma(n, 1)

print([n for n in srange(1, 150) if µ(s1(n)) != ν(s1(n))])

[3, 7, 21, 22, 31, 81, 93, 102, 110, 127, 142]


<p style="color:brown;font-size:large">A059404 are the numbers with different exponents in their prime factorizations.

In [74]:
def isA059404(n: int) -> bool: 
    return 1 < len(set(valuation(n, p) for p in prime_divisors(n)))

F = [n for n in range(1, 4362) if isA059404(n)]
#print(F)
set([ν(n) for n in F])

{0}

<p style="color:brown;font-size:large">A053810 are the numbers of the form p^e where both p and e are prime numbers.

In [75]:
def isA053810(n: int) -> bool:
    p = prime_divisors(n)
    return len(p) == 1 and is_prime(valuation(n, p[0]))

F = [n for n in srange(1, 2000) if isA053810(n)]
set([ν(n) for n in F])

{1}

<p style="color:brown;font-size:large">A217261 are the numbers of the form i^j^k, for i, j, k > 1. 

See also A372405, the exponentially powerful numbers whose prime <br>
factorization exponents are all powerful numbers > 1.

In [76]:
MAX = 10001
#MAX = 1185922
#MAX = 100000000

A217261List = sorted(set([x for x in [i^(j^2) 
              for j in range(2, 120) 
              for i in range(2, 120)] 
              if x < MAX]))

print(A217261List)

[16, 81, 256, 512, 625, 1296, 2401, 4096, 6561, 10000]


In [77]:
i=1
for n in A217261List[:12] :
    print([i], n, µ(n), sqrt(n))
    i += 1

[1] 16 0 4
[2] 81 0 9
[3] 256 0 16
[4] 512 0 16*sqrt(2)
[5] 625 0 25
[6] 1296 0 36
[7] 2401 0 49
[8] 4096 0 64
[9] 6561 0 81
[10] 10000 0 100


<h1 style="color:#CD5C5C;background:white; line-height: 150%;
border-top: thick solid #CD5C5C; float: left; width: 100%; margin-top: 1em;">
Appendix</h1>

In [78]:
%%time

[ν(n) for n in range(1, 20000)];

CPU times: user 999 ms, sys: 30 ms, total: 1.03 s
Wall time: 1.06 s


In [79]:
%%time

[µ(n) for n in range(1, 20000)];

CPU times: user 141 ms, sys: 0 ns, total: 141 ms
Wall time: 144 ms


<p style="color:brown;font-size:large">How fast can we compute µ (or ν)?</p>

* Harald A. Helfgott and Lola Thompson, Summing mu(n): a faster elementary algorithm, arXiv:2101.08773 [math.NT], 2021.

* Greg Hurst, Computations of the Mertens function and improved bounds on the Mertens conjecture, arXiv:1610.08551 [math.NT], 2016-2017.

The algorithm below is from:

* Marc Deléglise and Joöl Rivat, Computing the summation of the Möbius function, Experiment. Math. 5(4): 291-295 (1996).

In [80]:
def µList(x: int) -> list[int]:
    U = [1] * (x + 1)
    for p in prime_range(sqrt(x)):
        for n in range(1, x + 1):
            if n % p^2 == 0: 
                U[n] = 0
            elif n % p == 0: 
                U[n] = -p * U[n]

    for n in range(2, x + 1):
        u = U[n]
        if u != 0:
            U[n] = sign(u) if abs(u) == n else -sign(u)

    return U[1:]  

print(µList(20))
print([moebius(n) for n in range(1, 21)])

[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0]
[1, -1, -1, 0, -1, 1, -1, 0, 0, 1, -1, 0, -1, 1, 1, 0, -1, 0, -1, 0]


<p style="color:brown;font-size:large">About the Mertens function:</p>

* A nice blog post from Cleve Moler about the Mertens function with a soundtrack from the OEIS! 
https://blogs.mathworks.com/cleve/2024/10/22/mobius-mertens-and-redheffer/

* Holly Krieger discusses Merterns' Conjecture in a Numberphile video:
https://www.youtube.com/watch?v=uvMGZb0Suyc

<p style="color:brown;font-size:large">Some statistics about the distribution of -1, 0, and 1 for the Moebius function and the Mertens function.

In [81]:
c0 = c1 = c2 = 0
for n in range(1, 21001):
    µn = µ(n)
    if µn == 0: c0 +=1
    elif µn == 1: c1 +=1
    else: c2 += 1
    if n % 3000 == 0:
        print(n, c0, c1, c2, c1 - c2)

3000 1176 909 915 -6
6000 2354 1823 1823 0
9000 3527 2737 2736 1
12000 4707 3654 3639 15
15000 5880 4555 4565 -10
18000 7055 5467 5478 -11
21000 8231 6403 6366 37


<p style="color:brown;font-size:large"><p style="color:brown;font-size:large">Some statistics about the distribution of -1, 0, and 1 for the Hensel function and its summatory function.

In [82]:
c0 = c1 = c2 = 0
for n in range(1, 21001):
    vn = ν(n)
    if vn == 0: c0 +=1
    elif vn == 1: c1 +=1
    else: c2 += 1
    if n % 3000 == 0:
        print(n, c0, c1, c2, c1 - c2)

3000 1127 938 935 3
6000 2287 1861 1852 9
9000 3446 2781 2773 8
12000 4615 3705 3680 25
15000 5779 4610 4611 -1
18000 6943 5526 5531 -5
21000 8113 6465 6422 43


<p style="color:brown;font-size:large">How to plot the diagrams with tikzcd?</p>

<pre>
\usepackage{quiver}

\[\begin{tikzcd}[row sep=large]
	&& {1,1,1,...} \\
	{T(n, k)} &&&& {V(n, k)} \\
	\\
	{M(n, k)} &&&& {H(n, k)} \\
	& \mu && \nu
	\arrow["{c_1}"{description}, from=2-1, to=1-3]
	\arrow["inv"', from=2-1, to=4-1]
	\arrow["{c_1}"{description}, from=2-5, to=1-3]
	\arrow["sign"{description}, from=2-5, to=2-1]
	\arrow["inv", from=2-5, to=4-5]
	\arrow[dotted, tail reversed, from=4-1, to=4-5]
	\arrow["{c_1}"{description}, from=4-1, to=5-2]
	\arrow["{c_1}"{description}, from=4-5, to=5-4]
	\arrow["trans"{description}, dashed, from=5-2, to=5-4]
\end{tikzcd}\]

\[\begin{tikzcd}[row sep=large]
	&& A000012 \\
	A113704 &&&& A382944 \\
	\\
	A363914 &&&& A382881 \\
	& A008683 && A382883
	\arrow["{col_1}"{description}, from=2-1, to=1-3]
	\arrow["inv"', from=2-1, to=4-1]
	\arrow["{col_1}"{description}, from=2-5, to=1-3]
	\arrow["sign"{description}, from=2-5, to=2-1]
	\arrow["inv", from=2-5, to=4-5]
	\arrow[dotted, tail reversed, from=4-1, to=4-5]
	\arrow["{col_1}"{description}, from=4-1, to=5-2]
	\arrow["{col_1}"{description}, from=4-5, to=5-4]
	\arrow["trans"{description}, dashed, from=5-2, to=5-4]
\end{tikzcd}\]
</pre>

<p style="color:brown;font-size:large">Kurt_Hensel, Father of Valuation Theory. 

https://www.icts.res.in/sites/default/files/lem2016-29-08-2016-Sudesh.pdf

https://en.wikipedia.org/wiki/Kurt_Hensel

https://en.wikipedia.org/wiki/P-adic_valuation
